# **Modelo de Recomendação**
Nesse notebok desenvolvido o modelo de recomendação de produtos com dados de usuários de Olist, que foram selecionados e gerados a partir da análise exploratória. Além do treinamento do modelo de recomendação, será validado seu desempenho, aplicação do melhor modelo para recomendar novos produtos.

**Resumo do Modelo** \
Como foi utilizado modelos de classificação, foi necessário gerar uma amostra negativa para auxiliar o modelo no melhor entendimento dos dados, já que a base consiste apenas de compras realizadas. O tipo de amostragem negativa aplicada foi de similaridade, levando em consideração a similaridade de produtos que os clientes já compraram. \
Após a geração da amostragem negativa, foram realizados tratamentos distintos para valores vazios que existiam em colunas categóricas e numéricas, considerando a quantidade de valores vazios que existiam em cada coluna. \
Para o treinamento foram utilizados os modelos de Random Forest, XGB, LGBM e CatBoost, e foi avalidado o desempenho de cada um, sendo o **CatBoost** o melhor na configuração atual para recomendar produtos. \
Após todo o processo de treinamento e escolhado do melhor modelo, foram selecionados os clientes que seriam impactados pela as ações de marketing e aplicados para o modelo prever os melhores produtos para esses clientes. Ao final foi gerando um arquivo com o id do cliente, id do produto, categoria do produto e o score de possibilidade de achar aquele produto interessante para uso nas ações de marketing.

In [2]:
#Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("session").getOrCreate()
spark.conf.set("spark.sql.adaptive.enable", "true")

In [3]:
dataset = spark.read.option("header", True).parquet("dataset.parquet")
dataset.show(5)

+--------------------+--------------------+--------------------+--------------------+-------------+--------------+------+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|            order_id|          product_id|         customer_id|  customer_unique_id|customer_city|customer_state| price|freight_value|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|review_score|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------+------+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+
|00048cc3ae777c65d...|ef92defde845ab845...|816cbea969fe5b689...|85c835d128beae5

In [4]:
#Selecionando características de produtos e usuários
produto_features = ["price", "freight_value", "product_category_name", 
                    "product_name_lenght", "product_description_lenght", "product_photos_qty",
                    "product_weight_g", "product_length_cm", "product_height_cm",
                    "product_width_cm"]
cliente_features = ["customer_city", "customer_state"]